In [1]:
import os

%load_ext autoreload
%autoreload 2

""""
Devo code for moment by moment decoding --> i.e., for each time bin, find strength of representation of
each class of a variable (e.g., shape), as opposed to decoding average accuracy across trials.

Here collecting methods and quick analyses
"""

'"\nDevo code for moment by moment decoding --> i.e., for each time bin, find strength of representation of\neach class of a variable (e.g., shape), as opposed to decoding average accuracy across trials.\n\nHere collecting methods and quick analyses\n'

# Load a dataset

To load and plot a dataset of neural activity across population, in a PopAnal class object.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# (1) Load the dataset Diego 230630

In [4]:
from neuralmonkey.classes.population_mult import load_handsaved_wrapper

DFallpa = load_handsaved_wrapper()


In [5]:
DFallpa

,which_level,event,bregion,twind,pa,pa_x_shape
0,stroke,00_stroke,M1,"(-0.8, 0.8)",<neuralmonkey.classes.population.PopAnal objec...,"(62, 1947, 160)"
1,stroke,00_stroke,PMv,"(-0.8, 0.8)",<neuralmonkey.classes.population.PopAnal objec...,"(64, 1947, 160)"
2,stroke,00_stroke,PMd,"(-0.8, 0.8)",<neuralmonkey.classes.population.PopAnal objec...,"(51, 1947, 160)"
3,stroke,00_stroke,dlPFC,"(-0.8, 0.8)",<neuralmonkey.classes.population.PopAnal objec...,"(31, 1947, 160)"
4,stroke,00_stroke,vlPFC,"(-0.8, 0.8)",<neuralmonkey.classes.population.PopAnal objec...,"(58, 1947, 160)"
5,stroke,00_stroke,FP,"(-0.8, 0.8)",<neuralmonkey.classes.population.PopAnal objec...,"(59, 1947, 160)"
6,stroke,00_stroke,SMA,"(-0.8, 0.8)",<neuralmonkey.classes.population.PopAnal objec...,"(58, 1947, 160)"
7,stroke,00_stroke,preSMA,"(-0.8, 0.8)",<neuralmonkey.classes.population.PopAnal objec...,"(55, 1947, 160)"


In [15]:
pa = DFallpa["pa"].values[0]
dflab = pa.Xlabels["trials"]
dflab["shape_semantic"].value_counts()

shape_semantic
line-LL-LL          163
line-UU-UU          133
arcdeep-RR-RR       129
circle-XX-XX        125
usquare-UU-UU       123
line-UR-UR          115
arcdeep-DD-DD       114
line-UL-UL          108
Lcentered-DR-DR     104
Lcentered-UR-UR      88
zigzagSq-LL-1.0      80
V-UU-UU              76
Lcentered-UL-UL      74
arcdeep-UU-UU        71
Lcentered-DL-DL      69
usquare-RR-RR        60
V-RR-RR              50
zigzagSq-LL-0.0      49
squiggle3-LL-0.0     45
usquare-DD-DD        44
squiggle3-LL-1.0     41
zigzagSq-UU-0.0      36
V-DD-DD              27
squiggle3-UU-0.0     22
zigzagSq-UU-1.0       1
Name: count, dtype: int64

# Preprocessing

### FR Normalization

In [ ]:
from neuralmonkey.classes.population_mult import dfallpa_preprocess_fr_normalization
fr_normalization_method = "each_time_bin"
# fr_normalization_method = "across_time_bins"
plot_savedir = "/tmp"
dfallpa_preprocess_fr_normalization(DFallpa, fr_normalization_method, plot_savedir)

### Factorize all relevant labels FIRST here.


In [ ]:
if False: # Ingore this for now
    from neuralmonkey.analyses.decode_good import preprocess_factorize_class_labels_ints
    MAP_LABELS_TO_INT = preprocess_factorize_class_labels_ints(DFallpa)

### Pull out PMv data for now

In [ ]:
bregions = ["PMv"]
events = ["03_samp"]

DFallpa = DFallpa[(DFallpa["bregion"].isin(bregions)) & (DFallpa["event"].isin(events))].reset_index(drop=True)
PA =DFallpa["pa"].values[0]
PA.plotNeurHeat(trial=100)



# Relevant behavioral labels

In [ ]:
dflab = PA.Xlabels["trials"]

# -- Image features
# Each trial can have 1,2,3,or 4 strokes in the image
print(dflab["FEAT_num_strokes_task"].value_counts())

# Each trial has a specific set of locations that the shapes are presented:
print(dflab["taskconfig_loc"].value_counts())

# Each trial has a specific set of shapes presented
print(dflab["taskconfig_shp"].value_counts())

# Each trial has a specific set of (location x shapes) presented
print(dflab["taskconfig_shploc"].value_counts())



In [ ]:
# - Behavior features

# What shape did he do first, second, third, etc
# NOTE: some trials he does fewer shapes than exist on screen -- those cases he was sloppy in stroke quality, and he was aborted
dflab["seqc_0_shape"]
dflab["seqc_1_shape"]
dflab["seqc_2_shape"]
dflab["seqc_3_shape"]

# and onset location...
dflab["seqc_0_loc"]


In [ ]:
# This is a location config that is "ambiguous" in that on diff trials he will start on variable lcoations, iether (-1,0) or (1,1)

# These are the trials
dflab[dflab["taskconfig_loc"] == ((-1,0), (-1,1), (1,1))]

# Locations are variable
dflab[dflab["taskconfig_loc"] == ((-1,0), (-1,1), (1,1))]["seqc_0_loc"].value_counts()


# Decoding --> on each trial, can we decode him thinking about the shapes at locations (-1,0) and (1,1)?

##### Train decoder (fiurst Extract data for pre-samp and post-samp)

In [ ]:
# Train decoder on trials that only have one stroke..

inds = dflab[dflab["FEAT_num_strokes_task"]==1].index.tolist()
PAtrain = PA.slice_by_dim_indices_wrapper("trials", inds, reset_trial_indices=True)


In [ ]:
from neuralmonkey.analyses.decode_moment import Decoder
var = "seqc_0_shape" # Decoded variable
twind_train = [0.1, 0.7]

Dc = Decoder(PAtrain, var, twind_train)
Dc.train_decoder(PLOT=True)


In [ ]:
trial = 100
Dc.plot_single_trial(trial)

##### Get a single scalar score (by averaging the decode across time bins)


In [ ]:
list_twind = [
    [-0.7, -0.1],
    [0.1, 0.7],
]

scores = Dc.scalar_score_twinds_trials(list_twind)

##### Average the decode over trials, to return a single row per trial class

In [ ]:
# Returns array: (ntrial classes, n decode clasess, ntwinds)

In [ ]:
vars_trial = ["seqc_0_shape"]

Dc.scalar_score_twinds_trialgroupings(vars_trial, list_twind, PLOT=True)

In [ ]:
Dc.PAtrain

# Testing the hypothesis

In [ ]:
# PRint trials with > 1 stroke
dflab = PA.Xlabels["trials"]
dflab[dflab["FEAT_num_strokes_task"]>1].index.tolist()

In [ ]:
# Plot a multi-stroke trial 
# NOTE: you can see here switching between representations of shapes

trial = 295

for i in [0,1,2,3]:
    print("stroke", i, " -- ", dflab.iloc[trial][f"seqc_{i}_shape"])

tbin_dur=0.15
tbin_slide=0.01
Dc.plot_single_trial(trial, PA=PA, tbin_dur=tbin_dur, tbin_slide=tbin_slide)


#### plot after sort labels (in progress)


In [ ]:
# NOTE: this is written for a very specific expeirmetn (Novel vs. orig prims)_.

In [ ]:
def F(x):
    if "novelprim" in x:
        return f"2{x}"
    elif "presamp" in x:
        return f"3{x}"
    else:
        return f"1{x}"


In [ ]:
labs_rows = [x[0] for x in grpdict.keys()]
labs_for_sorting = [F(x) for x in labs_rows]
inds = np.argsort(labs_for_sorting)
print(labs_for_sorting)
res_sorted = resthis[inds, :, :]
labs_rows_sorted = [labs_rows[i] for i in inds]

labs_cols = mlb.classes_
labs_for_sorting = [F(x) for x in labs_cols]
print(labs_for_sorting)
inds = np.argsort(labs_for_sorting)
res_sorted = res_sorted[:, inds, :]
labs_cols_sorted = [labs_cols[i] for i in inds]

from pythonlib.tools.snstools import heatmap_mat

ncols = 3
nrows = int(np.ceil(len(list_twind)/ncols))
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*7, nrows*5))

for ax, ind_twind in zip(axes.flatten(), range(len(list_twind))):
    if True:
        heatmap_mat(res_sorted[:, :, ind_twind], ax, False, zlims=(0, 0.3))    
        ax.set_xlabel("decoded class label")
        ax.set_ylabel("trial label")
        ax.set_title(f"twind: {list_twind[ind_twind]}")
    else:
        res_vec = np.mean(res_sorted[:, :, ind_twind], axis=0)
        xlab = [f"{i}-{x[:1]}" for i, x in enumerate(labs_cols_sorted)]
        ax.plot(xlab, res_vec, "ok")

        from pythonlib.tools.plottools import rotate_x_labels
        rotate_x_labels(ax)
        ax.set_xlabel("decoded class label")
        ax.set_ylabel("trial label")
        ax.set_title(f"twind: {list_twind[ind_twind]}")
        ax.set_ylim([0, 0.1])

        

# Older methods

### Extract single trial results

In [ ]:

# PARAMS
var_decode = "seqc_0_shape"
bregion = "PMv"
twind = (-0.4, 0.6)

In [ ]:

a = DFRES["var_decode"]==var_decode
b = DFRES["bregion"]==bregion
c = DFRES["twind"]==twind
dfthis = DFRES[a & b & c]

times = dfthis["time"]
import numpy as np
labels_predicted = np.stack(dfthis["labels_predicted"]).T # (ntrials, ntimes)
labels_test = np.stack(dfthis["labels_test"]).T
conf_scores = np.transpose(np.stack(dfthis["conf_scores"]), [1,0,2]) # (ntrials, ntimes, nclasses)

# score each time bin as correct or incorrect
labels_correct = (labels_predicted == labels_test).astype(int)


In [ ]:
# For each trial, what was its label
assert np.all(np.diff(labels_test, axis=1))==0, "otherwise cant do next step."
labels_each_trial = labels_test[:,0]
labels_orig = PA_test.Xlabels["trials"][var_decode].tolist()
assert len(labels_orig)==len(labels_each_trial)

map_labint_to_trials = {}
map_labint_to_laborig = {}
labels_unique = np.unique(labels_each_trial)
for lab in labels_unique:
    inds_this_label = np.argwhere(labels_each_trial == lab).squeeze() # the indices which ahve this label as the CORRECT
    map_labint_to_trials[lab] = inds_this_label

    # Map it back to original label
    lab_orig = PA_test.Xlabels["trials"][var_decode][inds_this_label].unique()
    assert len(lab_orig)==1
    map_labint_to_laborig[lab] = lab_orig[0]

map_trial_to_labint = {i:labint for i, labint in enumerate(labels_each_trial)}
print(map_labint_to_laborig)    
map_laborig_to_labint = {}
for labint, laborig in map_labint_to_laborig.items():
    assert laborig not in map_laborig_to_labint
    map_laborig_to_labint[laborig] = labint

import matplotlib.pyplot as plt
from pythonlib.tools.plottools import makeColors
pcols = makeColors(len(map_labint_to_laborig))
map_trial_to_seq = {}
map_trial_to_seq_ints = {}
for i, seq in enumerate(PA_test.Xlabels["trials"].loc[:, ["seqc_0_shape", "seqc_1_shape"]].values.tolist()):
    map_trial_to_seq[i] = (tuple(seq))
    map_trial_to_seq_ints[i] = [map_laborig_to_labint[s] if s in map_laborig_to_labint else s for s in seq]


In [ ]:
# Pick n random trials
import random
n=  9
trials_all = list(range(len(map_trial_to_labint)))
trials = random.sample(trials_all, n)
ncols = 3
nrows = int(np.ceil(len(trials)/ncols))
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*4, nrows*3), sharex=True, sharey=True)

for tr, ax in zip(trials, axes.flatten()):
    
    # ax = axes.flatten()[0]
    lab_pred = labels_predicted[tr,:]
    # lab_corr = labels_correct[trial,:]
    # lab_test = labels_test[trial,:]
    # ax.plot(times, lab_pred, "-ok"),
    # ax.plot(times, lab_test, "-r")
    
    # for each class, plot its time series
    for labint in map_labint_to_laborig:
        cs_this_lab = conf_scores[tr, :, labint].squeeze()
        
        ax.plot(times, cs_this_lab, label=labint, color=pcols[labint])
        # ax.plot(times[lab_pred==labint], cs_this_lab[lab_pred==labint], "-", color=pcols[labint], linewidth=4)
        ax.plot(times[lab_pred==labint], cs_this_lab[lab_pred==labint], "s", color=pcols[labint])
    
    ax.axvline(0, color="k", alpha=0.5)
    ax.set_title(f"{var_decode}={map_trial_to_labint[tr]}", color=pcols[map_trial_to_labint[tr]])
    ax.set_xlabel(f"seq: {map_trial_to_seq_ints[tr]}")
    ax.legend()
        

In [ ]:
# For each class label, collect all trials for which that is the correct label, and overlap

labels_unique = np.unique(labels_each_trial)
ncols = 3
nrows = int(np.ceil(len(labels_unique)/ncols))

fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*4, nrows*4))

for lab, ax in zip(labels_unique, axes.flatten()):
    inds_this_label = np.argwhere(labels_each_trial == lab).squeeze() # the indices which ahve this label as the CORRECT 
    ax.plot(times, labels_predicted[inds_this_label, :].T, "-ok", alpha=0.05)
    ax.set_title(lab)
    
    

In [ ]:
fig, axes = plt.subplots(2,2, figsize=(10,10))

ax = axes.flatten()[0]
ax.plot(times, labels_correct.T, "xk", alpha=0.003);

In [ ]:
##### Plot the results, comparing score across methods!!
%matplotlib inline
import seaborn as sns
fig = sns.relplot(data=DFRES, x="time", y="score", hue="bregion", col="var_decode",  kind="line")


In [ ]:
##### Plot the results, comparing score across methods!!
%matplotlib inline
import seaborn as sns
fig = sns.relplot(data=DFRES, x="time", y="score", hue="bregion", col="var_decode",  kind="line")


In [ ]:
from neuralmonkey.analyses.decode_good import decodewrapouterloop_categorical_cross_time